In [1]:
cd ..

c:\Users\14102\Brown\Realizing_Rights\RealizingRights


In [2]:
from utilities.web_assist import recurse_scan_all_unique_links_in_site, closest_link_match, make_https, prepend_root_to_url, make_driver_utils
from url_scraper.link_data import LinkData
import pandas as pd

In [3]:
district_df = pd.read_excel(io='data/USSchoolDistrictWebsiteInfo.xlsx', sheet_name='ELSI Export', header=6)
district_df['URL'] = district_df['URL'].apply(lambda x: make_https(url=x))

district_df['NumStudents'] = pd.to_numeric(district_df['NumStudents'], errors='coerce')
district_df_with_urls = district_df[~district_df['URL'].isna()]
district_df_with_urls['URL'] = district_df_with_urls['URL'].apply(lambda x: prepend_root_to_url(base_url=x, prefix=''))
district_df_with_urls = district_df_with_urls[district_df_with_urls['NumStudents'] > 0]
district_df_with_urls = district_df_with_urls[district_df_with_urls['NumSchools'] > 0].reset_index(drop=True)

c:\Users\14102\anaconda3\envs\real_right_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [4]:
drvr, actions, wait = make_driver_utils()

blacklist_terms = ['login', 'aspx', 'lightbox', 'file', '.php', '#', '.pdf', 'doc', 'jpg', 'tel:']

link_keywords = ['boe meeting', 'board meeting', 'board of education', 'parent', 
                 'meeting', 'video recording', 'board video recording', 'superintendent']

board_meeting_keywords = ['boe meeting', 'board meeting', 'boe recording', 'board recording',
                            'boe meeting recording', 'board meeting recording', 'boe meeting video',
                            'board meeting video', 'boe video recording', 'board video recording']
url_data = {}
for url in district_df_with_urls['URL']:
    start_link_set = set()
    start_link_set.add(LinkData(link_text='BASE', link_url=url, depth=0))
    recursed_lcl_links, recursed_ext_links = set(), set()
    recursed_lcl_links, recursed_ext_links = recurse_scan_all_unique_links_in_site(url=url,
                                                                                    base_url=url, 
                                                                                    local_link_set=start_link_set,
                                                                                    external_link_set=set(),
                                                                                    drvr=drvr, 
                                                                                    actions=actions,
                                                                                    blacklist_terms=blacklist_terms,
                                                                                    wait=wait,
                                                                                    link_keywords=link_keywords)

    cur_sim = 50
    best_link = None
    best_lcl_link = None
    for lcl_link in recursed_lcl_links:
        sim = closest_link_match(lcl_link.link_text, board_meeting_keywords)
        if sim > cur_sim:
            best_lcl_link = lcl_link
            sim = cur_sim

    # Identify External Links Pointing to social media sites 
    sites_identified = {}
    ext_link_list = list(recursed_ext_links)
    num_ext_links = len(ext_link_list)
    print(recursed_ext_links)
    ext_id = 0
    social_media_sites = ['youtube', 'vimeo', 'facebook', 'twitter']
    while ext_id < num_ext_links:
        ext_link = ext_link_list[ext_id]
        scl_id = 0
        while scl_id < len(social_media_sites):
            if social_media_sites[scl_id] in ext_link.link_url:
                sites_identified[social_media_sites.pop(scl_id)] = ext_link.depth_found
            scl_id += 1

        ext_id += 1

        if len(social_media_sites) == 0:
            break

    url_data[url] = (sites_identified, best_lcl_link)
    print(sites_identified, best_lcl_link)



Depth 0 for https://www.geoacademies.org
{ENROLL: https://www.garymiddlecollege.org, None: https://www.facebook.com/GEOFoundation, None: https://www.linkedin.com/company/geo-foundation-academies, ENROLL: https://www.geofocusacademy.org, : https://www.geonextgenhsindy.org, : https://www.21cchartergary.org, 3145 N. Meridian St.Indianapolis, IN 46208: https://www.goo.gl/maps/6ntxMaogCdMBAyqp6, ENROLL: https://www.21cchartergary.org/our-approach/enroll-your-child, None: https://www.twitter.com/GEOFoundation, : https://www.garymiddlecollege.org/gmc-west}
{'facebook': 0, 'twitter': 0} None
set()
{} None
Depth 0 for https://www.21stprepschool.org
set()
{} None
Depth 0 for https://www.beattietech.com
{Facebook: https://www.facebook.com/awbeattiecc, Powered By: https://www.schoolpointe.com, Translate: https://www.translate.google.com, Instagram: https://www.instagram.com/awbeattiecareercenter, Click here for a feature video of Nadia Allen in Surgical Sciences: https://www.youtube.com/watch, You

In [ ]:
for url in district_df_with_urls['URL']:
    print(url)
    break